In [3]:
import pandas as pd
import numpy as np
import os

In [7]:
df = pd.read_csv("ng_messy_sales.csv")

print(df.shape)
df.head()

(550, 9)


,Customer Name,State,Product,Units Sold,Unit Price,Total Sale,Sale Date,Sales Channel,Order ID
0,NaN,rivers,KEYBOARD,NaN,NaN,NaN,2025-04-27,Online,NaN
1,Allison Hill,Lagos,Headphones,NaN,267992.94,NaN,2024-03-15,Wholesale,4c636e95-025f-4543-8997-623ae0723d96
2,Noah Rhodes,Anambra,Keyboard,NaN,42364.41,NaN,2024-12-10,NaN,edaf3766-1b78-4ede-9a4f-fc0c9165f2ed
3,Angie Henderson,Delta,Keyboard,NaN,279444.94,NaN,2024-04-05,NaN,74503887-48d9-4846-95c5-51fcfba57cc8
4,Daniel Wagner,Delta,Tablet,NaN,95899.74,NaN,2025-01-12,NaN,8639bd41-8b15-4d94-a42d-0cd7fd359f6a


In [8]:
missing_counts = df[["Units Sold", "Unit Price", "Total Sale"]].isna().sum()
print(missing_counts)


Units Sold    395
Unit Price     55
Total Sale    413
dtype: int64


In [9]:
duplicate_count = df["Order ID"].duplicated().sum()
print("Duplicate Order IDs:", duplicate_count)


Duplicate Order IDs: 39


Dropping the rows where the Units Sold, Unit Price, and Total Sale are ALL missing

In [10]:
df = df.dropna(subset=["Units Sold", "Unit Price", "Total Sale"], how="all")



Drop duplicates Order ID

In [11]:
df = df.drop_duplicates(subset=["Order ID"])


In [12]:
df.head()

,Customer Name,State,Product,Units Sold,Unit Price,Total Sale,Sale Date,Sales Channel,Order ID
1,Allison Hill,Lagos,Headphones,NaN,267992.94,NaN,2024-03-15,Wholesale,4c636e95-025f-4543-8997-623ae0723d96
2,Noah Rhodes,Anambra,Keyboard,NaN,42364.41,NaN,2024-12-10,NaN,edaf3766-1b78-4ede-9a4f-fc0c9165f2ed
3,Angie Henderson,Delta,Keyboard,NaN,279444.94,NaN,2024-04-05,NaN,74503887-48d9-4846-95c5-51fcfba57cc8
4,Daniel Wagner,Delta,Tablet,NaN,95899.74,NaN,2025-01-12,NaN,8639bd41-8b15-4d94-a42d-0cd7fd359f6a
5,Cristian Santos,Oyo,Camera,NaN,118189.90,NaN,2023-08-30,Direct,225aed6c-f045-4043-a3a6-8a707e710b55


In [13]:
df.tail()

,Customer Name,State,Product,Units Sold,Unit Price,Total Sale,Sale Date,Sales Channel,Order ID
544,Mr. Justin Green III,Sokoto,KEYBOARD,65.0,206948.51,13451653.15,2023-12-02,Online,ec93e596-2b01-4723-8cf6-97a57603c36c
545,Zachary Mitchell,Ekiti,Monitor,6.0,23989.09,143934.54,2025-07-07,Wholesale,b9744cde-d259-4960-aee3-560792ea1324
547,Katherine Martinez,Bauchi,Phone,NaN,104084.92,NaN,2025-05-18,Retail,61fa8e7a-078c-46ce-bf02-23bc651ea252
548,Jodi Roach,Niger,Charger,NaN,29756.89,NaN,2025-06-29,Direct,5a9c0679-2c3d-47e2-ad39-da75aee70ab2
549,Brandon Fleming,imo,Camera,NaN,70027.13,NaN,2024-02-20,NaN,46653497-2fe9-42aa-a886-a01136f7a26c


I noticed product name and state both have standardized lower & uppercses

In [14]:
df["Product"] = df["Product"].str.strip().str.title()

df["State"] = df["State"].str.strip().str.title()


In [15]:
df.head()
df.info()
df.describe()


<class 'pandas.core.frame.DataFrame'>
Index: 480 entries, 1 to 549
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Customer Name  444 non-null    object 
 1   State          480 non-null    object 
 2   Product        480 non-null    object 
 3   Units Sold     146 non-null    float64
 4   Unit Price     464 non-null    float64
 5   Total Sale     130 non-null    float64
 6   Sale Date      480 non-null    object 
 7   Sales Channel  386 non-null    object 
 8   Order ID       479 non-null    object 
dtypes: float64(3), object(6)
memory usage: 37.5+ KB


,Units Sold,Unit Price,Total Sale
count,146.000000,464.000000,1.300000e+02
mean,47.349315,155148.837069,6.976575e+06
std,26.639766,84955.415949,6.234590e+06
min,1.000000,1403.130000,5.788704e+04
25%,26.000000,83773.965000,1.841694e+06
50%,48.000000,157565.060000,4.947761e+06
75%,67.750000,226426.175000,1.105162e+07
max,98.000000,298048.290000,2.873459e+07


The sale date is still a string, i'll need to convert it to a date time 

In [16]:
df["Sale Date"] = pd.to_datetime(df["Sale Date"], errors="coerce")


Due to the missing values and inconsistencies in the data for Units Sold,Unit Price and Total Sale
I'll have to validate and clean the data

In [17]:
df["Calculated Sale"] = df["Units Sold"] * df["Unit Price"]
df["Diff"] = df["Total Sale"] - df["Calculated Sale"]


In [18]:

print("Rows with mismatched totals:", (df["Diff"].abs() > 1).sum())

Rows with mismatched totals: 0


In [19]:
df.head(10)

,Customer Name,State,Product,Units Sold,Unit Price,Total Sale,Sale Date,Sales Channel,Order ID,Calculated Sale,Diff
1,Allison Hill,Lagos,Headphones,NaN,267992.94,NaN,2024-03-15,Wholesale,4c636e95-025f-4543-8997-623ae0723d96,NaN,NaN
2,Noah Rhodes,Anambra,Keyboard,NaN,42364.41,NaN,2024-12-10,NaN,edaf3766-1b78-4ede-9a4f-fc0c9165f2ed,NaN,NaN
3,Angie Henderson,Delta,Keyboard,NaN,279444.94,NaN,2024-04-05,NaN,74503887-48d9-4846-95c5-51fcfba57cc8,NaN,NaN
4,Daniel Wagner,Delta,Tablet,NaN,95899.74,NaN,2025-01-12,NaN,8639bd41-8b15-4d94-a42d-0cd7fd359f6a,NaN,NaN
5,Cristian Santos,Oyo,Camera,NaN,118189.90,NaN,2023-08-30,Direct,225aed6c-f045-4043-a3a6-8a707e710b55,NaN,NaN
6,Connie Lawrence,Rivers,Tablet,36.0,185141.90,6665108.40,2024-09-10,Wholesale,45d8a6ad-77d0-459e-8114-14f8d9df0d0e,6665108.40,9.313226e-10
7,Abigail Shaffer,Imo,Keyboard,NaN,252859.28,NaN,2024-09-04,Online,54a7b69b-1cd6-4b09-8f0e-6d2b315c167a,NaN,NaN
8,Gina Moore,Kano,Camera,NaN,178320.53,NaN,2024-02-21,Wholesale,a5f60735-757b-4b29-bb2a-a1462999394c,NaN,NaN
9,Gabrielle Davis,Plateau,Tablet,NaN,152440.84,NaN,2025-06-02,Direct,039b058c-2faa-4e0b-b7f6-0cdf41dc1c60,NaN,NaN
11,Monica Herrera,Abuja,Laptop,3.0,153159.37,459478.11,2025-05-02,Retail,317304b5-c1dd-484a-aca5-8d95915a1c30,459478.11,0.000000e+00


In [20]:
df["Year"] = df["Sale Date"].dt.year
df["Month"] = df["Sale Date"].dt.month
df["Quarter"] = df["Sale Date"].dt.to_period("Q")


In [31]:
print(df[["Sale Date", "Year", "Month", "Quarter"]].head())


   Sale Date  Year  Month Quarter
1 2024-03-15  2024      3  2024Q1
2 2024-12-10  2024     12  2024Q4
3 2024-04-05  2024      4  2024Q2
4 2025-01-12  2025      1  2025Q1
5 2023-08-30  2023      8  2023Q3


In [21]:
df["Revenue Per Unit"] = df["Unit Price"]

In [32]:
print(df[["Product", "Unit Price", "Revenue Per Unit"]].head())


      Product  Unit Price  Revenue Per Unit
1  Headphones   267992.94         267992.94
2    Keyboard    42364.41          42364.41
3    Keyboard   279444.94         279444.94
4      Tablet    95899.74          95899.74
5      Camera   118189.90         118189.90


In [22]:
customer_freq = df["Customer Name"].value_counts()

In [34]:
print(customer_freq.head(5))

Customer Name
Matthew Moore       2
Allison Hill        1
Victor Brown        1
Theresa Williams    1
Kayla Rodriguez     1
Name: count, dtype: int64


In [23]:
product_revenue = df.groupby("Product")["Total Sale"].sum().sort_values(ascending=False)
product_units = df.groupby("Product")["Units Sold"].sum().sort_values(ascending=False)


In [27]:
print(product_revenue)
print(product_units)

Product
Keyboard      1.783906e+08
Phone         1.676139e+08
Headphones    1.426256e+08
Tablet        1.405429e+08
Monitor       1.144702e+08
Charger       1.030686e+08
Laptop        3.514419e+07
Camera        2.509875e+07
Name: Total Sale, dtype: float64
Product
Keyboard      1330.0
Headphones    1053.0
Tablet        1052.0
Phone          965.0
Monitor        846.0
Charger        693.0
Camera         531.0
Laptop         443.0
Name: Units Sold, dtype: float64


In [24]:
state_revenue = df.groupby("State")["Total Sale"].sum().sort_values(ascending=False)


In [28]:
print(state_revenue)

State
Benue          83651542.02
Rivers         80846826.76
Delta          70752955.06
Kano           66430447.61
Imo            63466338.88
Plateau        61414056.17
Kaduna         58834730.64
Niger          54765574.51
Sokoto         44669534.61
Lagos          44548407.02
Katsina        36094558.16
Enugu          34518737.93
Bauchi         33982869.24
Cross River    33914235.67
Abuja          33383113.98
Oyo            28967495.85
Osun           26309925.52
Borno          23286853.24
Ekiti          16016820.99
Anambra        11099688.88
Name: Total Sale, dtype: float64


In [25]:
channel_revenue = df.groupby("Sales Channel")["Total Sale"].sum().sort_values(ascending=False)
channel_units = df.groupby("Sales Channel")["Units Sold"].sum().sort_values(ascending=False)


In [29]:
print(channel_revenue)
print(channel_units)


Sales Channel
Online       2.294837e+08
Retail       1.956314e+08
Wholesale    1.654142e+08
Direct       1.523044e+08
Name: Total Sale, dtype: float64
Sales Channel
Online       1669.0
Retail       1407.0
Wholesale    1366.0
Direct       1148.0
Name: Units Sold, dtype: float64


In [26]:
monthly_sales = df.groupby(df["Sale Date"].dt.to_period("M"))["Total Sale"].sum()

In [30]:
print(monthly_sales)

Sale Date
2023-07    17958791.92
2023-08    33113463.09
2023-09     6847245.49
2023-10    30724980.41
2023-11    38192829.56
2023-12    32504101.32
2024-01    33928727.92
2024-02    30259644.19
2024-03    80194387.81
2024-04    38331241.04
2024-05    37786995.73
2024-06     6292829.97
2024-07    85177032.40
2024-08    29512743.12
2024-09    72465359.50
2024-10     4024755.52
2024-11    15539825.64
2024-12    28489123.08
2025-01    42967401.97
2025-02    54262100.62
2025-03    51367343.92
2025-04    48131538.22
2025-05    25876060.48
2025-06    21359175.61
2025-07    41647014.21
Freq: M, Name: Total Sale, dtype: float64


In [35]:
print("1. Top products by revenue:", product_revenue.head(3))
print("2. State with highest revenue:", state_revenue.idxmax())
print("3. Channel comparison:", channel_revenue)
print("4. Peak sales month:", monthly_sales.idxmax())

1. Top products by revenue: Product
Keyboard      1.783906e+08
Phone         1.676139e+08
Headphones    1.426256e+08
Name: Total Sale, dtype: float64
2. State with highest revenue: Benue
3. Channel comparison: Sales Channel
Online       2.294837e+08
Retail       1.956314e+08
Wholesale    1.654142e+08
Direct       1.523044e+08
Name: Total Sale, dtype: float64
4. Peak sales month: 2024-07


In [36]:
df.to_csv("cleaned_sales_data.csv", index=False)